## Проект для Интернет-магазина с BERT

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Наша задача — обучить модель, которая будет классифицировать комментарии как **токсичные (1)** или **нетоксичные (0)**.  

**Целевая метрика**: F1 ≥ 0.75

## План проекта (расширенный с BERT):

**Цель:**  
Построить модель, которая автоматически определяет токсичность комментариев пользователей для платформы Викишоп. Это позволит модерировать контент и поддерживать здоровую коммуникацию на платформе.

---

Этапы проекта:

1. **Загрузка и предварительная подготовка данных**
   - Импорт библиотек
   - Загрузка и первичный анализ данных
   - Проверка на пропуски, дубликаты, баланс классов

2. **Очистка текста**
   - Приведение к нижнему регистру
   - Удаление лишних символов, HTML-тегов, стоп-слов и др.

3. **Базовое моделирование**
   - Векторизация текста с помощью `TfidfVectorizer`
   - Обучение моделей: `LogisticRegression`, `LinearSVC`
   - Подбор гиперпараметров с помощью `GridSearchCV`
   - Оценка качества моделей с использованием F1-меры

4. **Дообучение BERT (fine-tuning)**
   - Использование предобученной модели `bert-base-multilingual-cased`
   - Создание кастомного Dataset и DataLoader
   - Fine-tuning на части обучающей выборки
   - Оценка метрики F1 на валидационной выборке

5. **Сравнение моделей**
   - Сравнение F1-метрик базовых моделей и BERT
   - Анализ производительности (время, ресурсы)
   - Интерпретация результатов

6. **Выводы**
   - Обоснование выбора финальной модели
   - Рекомендации по внедрению и возможному масштабированию

---

**Метрика качества:**  
Основная метрика — F1-мера (взвешенное среднее между precision и recall), подходящая для несбалансированных классов.


## Импорты


In [1]:
# Базовые библиотеки
import re
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm

# Модели и метрики
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV,
    StratifiedKFold,
    StratifiedShuffleSplit
)
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
from sklearn.preprocessing import FunctionTransformer
from sklearn.utils.class_weight import compute_class_weight

# Transformers и PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    BertTokenizer,
    BertModel,
    BertForSequenceClassification,
    AdamW,
    get_scheduler
)

# Отключаем предупреждения
warnings.filterwarnings('ignore')

import nltk
from nltk.corpus import wordnet
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dmitrysergeenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/dmitrysergeenko/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dmitrysergeenko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Загрузка данных
df = pd.read_csv('/Users/dmitrysergeenko/Downloads/Яндекс.Практикум/Проекты на зачет/BERT. Векторизация текста/toxic_comments.csv')

df.info()

# Просмотр первых 5 строк
df.head()

# Распределение по классам (0 — нетоксичный, 1 — токсичный)
print("\nРаспределение классов:")
print(df['toxic'].value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB

Распределение классов:
toxic
0    143106
1     16186
Name: count, dtype: int64


В датасете содержится 159 292 комментария, каждый из которых размечен по признаку токсичности:

Столбец text содержит текст комментария.

Целевой признак toxic принимает значение 1 для токсичных комментариев и 0 для нетоксичных.

Распределение классов несбалансированное:

Нетоксичных комментариев — 143 106 (≈ 89.8%)

Токсичных — 16 186 (≈ 10.2%)

Также обнаружен лишний столбец Unnamed: 0, который является индексом и не несёт полезной информации - он будет удалён на этапе подготовки данных.

При обучении моделей будем учитывать несбалансированность классов: F1-мера будет использоваться как основная метрика качества, поскольку она сбалансировано учитывает и полноту, и точность при работе с неравными классами.



In [3]:
# Удалим лишнюю колонку
df = df.drop(columns=['Unnamed: 0'])

# Просмотрим первые 5 строк после удаления 
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Вывод: лишний столбец успешно удален.

Очистим текст (удалим спецсимволы и приведём к нижнему регистру)

In [4]:
# Очистим текст

def clean_text(text):
    text = text.lower()                             # приведение к нижнему регистру
    text = re.sub(r'\n+', ' ', text)                # замена переносов на пробел
    text = re.sub(r'http\S+', '', text)             # удаление URL
    text = re.sub(r'<.*?>', '', text)               # удаление HTML
    text = re.sub(r'@\w+', '', text)                # удаление @mention
    text = re.sub(r'[^a-zA-Zа-яА-Я0-9\s]', '', text)  # удаление знаков препинания
    text = re.sub(r'\s+', ' ', text).strip()        # удаление лишних пробелов
    return text

df['clean_text'] = df['text'].apply(clean_text)

## Вариант решения без BERT

Разделим на обучающую и тестовую выборки

Применим TfidfVectorizer

Обучим LogisticRegression

Посчитаем F1-метрику

In [5]:
# 1. Делим на train+val и test
X_temp, X_test, y_temp, y_test = train_test_split(
    df['clean_text'], df['toxic'], test_size=0.2, random_state=42, stratify=df['toxic'])

# 2. Делим X_temp на train и val
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)  # 0.25 * 0.8 = 0.2

# 3. TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

# 4. Обучение на train, настройка по val
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_tfidf, y_train)

# 5. Оценка на val
val_pred = model.predict(X_val_tfidf)
val_f1 = f1_score(y_val, val_pred)
print(f"F1 на валидации: {val_f1:.3f}")

# 6. Финальная оценка на test
test_pred = model.predict(X_test_tfidf)
test_f1 = f1_score(y_test, test_pred)
print(f"F1 на тесте: {test_f1:.3f}")


F1 на валидации: 0.678
F1 на тесте: 0.689


Промежуточный вывод:

Для классификации токсичных комментариев был реализован классический подход машинного обучения (модель без BERT):

Тексты были преобразованы в числовые векторы с помощью метода TF-IDF.

Была обучена модель логистической регрессии.

Для оценки использовалась F1-мера на тестовой выборке.

Результат:

F1-мера на тесте = 0.689

Модель пока не достигает требуемого уровня качества. 


Доработаем классический ML-подход чтобы достичь метрики по ТЗ включающий три модели:

LogisticRegression

LinearSVC


- Расширим настройки TF-IDF (настроем параметры TfidfVectorizer)
- Используем пайплайн
- Подберем наилучшую модель с оптимальными параметрами по F1-метрике.
- Учтем несбалансированность классов (повысим чувствительность к меньшинству -токсичные комментарии).

In [6]:
# 1. Делим: train+val и test
X_temp, X_test, y_temp, y_test = train_test_split(
    df['clean_text'], df['toxic'], stratify=df['toxic'], test_size=0.2, random_state=42
)

# 2. Задаём кросс-валидацию
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# 3. Пайплайн
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1, 3),
        max_df=0.9,
        min_df=3,
        max_features=50000,
        sublinear_tf=True
    )),
    ('clf', LogisticRegression())  # переопределяется
])

# 4. Сетка параметров
param_grid = [
    {
        'clf': [LogisticRegression(max_iter=1000, class_weight='balanced')],
        'clf__C': [0.5, 1.0, 2.0]
    },
    {
        'clf': [LinearSVC(class_weight='balanced', max_iter=1000)],
        'clf__C': [0.5, 1.0, 2.0]
    }
]

# 5. GridSearchCV на train+val
grid = GridSearchCV(pipeline, param_grid, scoring='f1', cv=cv, n_jobs=-1, verbose=2)
grid.fit(X_temp, y_temp)

# 6. Финальная оценка
print("Лучшая модель:", grid.best_estimator_['clf'])
print("Лучшие параметры:", grid.best_params_)

y_pred = grid.predict(X_test)
f1 = f1_score(y_test, y_pred)
print(f"\nF1-мера на тестовой выборке: {f1:.3f}")


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] END clf=LogisticRegression(class_weight='balanced', max_iter=1000), clf__C=1.0; total time=  42.1s
[CV] END clf=LogisticRegression(class_weight='balanced', max_iter=1000), clf__C=1.0; total time=  42.4s
[CV] END clf=LogisticRegression(class_weight='balanced', max_iter=1000), clf__C=1.0; total time=  42.2s
[CV] END clf=LogisticRegression(class_weight='balanced', max_iter=1000), clf__C=2.0; total time=  42.1s
[CV] END clf=LogisticRegression(class_weight='balanced', max_iter=1000), clf__C=0.5; total time=  42.9s
[CV] END clf=LogisticRegression(class_weight='balanced', max_iter=1000), clf__C=0.5; total time=  43.1s
[CV] END clf=LogisticRegression(class_weight='balanced', max_iter=1000), clf__C=0.5; total time=  43.2s
[CV] END clf=LogisticRegression(class_weight='balanced', max_iter=1000), clf__C=2.0; total time=  43.1s
[CV] END .clf=LinearSVC(class_weight='balanced'), clf__C=0.5; total time=  42.8s
[CV] END .clf=LinearSVC(cla

Итог обучения моделей:

Лучшая модель: LinearSVC(C=0.5, class_weight='balanced')

Лучшая F1-мера на тестовой выборке: 0.761

Пороговое значение F1 ≥ 0.75 выполнено. Это значит, что мы уже можем считать задачу успешно решённой (F1 ≥ 0.75).

Время обучения каждой модели в кросс-валидации: ~22–28 секунд

## Вариант решения с BERT


In [7]:
# 1. Выбираем тексты и целевой признак
texts = df['text'].astype(str).tolist()
labels = df['toxic'].values

# 2. Инициализируем токенизатор
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# 3. Токенизируем тексты
tokenized = [tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=512) for text in texts]

# 4. Находим максимальную длину токенизированного текста
max_len = max(len(seq) for seq in tokenized)

# 5. Паддинг до одинаковой длины
padded = np.array([seq + [0]*(max_len - len(seq)) for seq in tokenized])

# 6. Создаём attention mask (1 — реальные токены, 0 — паддинг)
attention_mask = np.where(padded != 0, 1, 0)


Мы получили:

padded — токенизированные тексты с паддингом;

attention_mask — маска для игнорирования паддинга;

labels — целевые значения (toxic).

Далее создадим эмбеддинги из текстов при помощи предобученной модели BERT.

In [8]:
# 1. Настраиваем устройство
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")

# 2. Загружаем предобученную модель BERT
model = BertModel.from_pretrained('bert-base-multilingual-cased')
model.to(device)
model.eval()

# 3. Преобразуем данные в тензоры и ограничиваем размер
N = 10000
input_ids = torch.tensor(padded[:N], dtype=torch.long).to(device)
attention_mask_tensor = torch.tensor(attention_mask[:N], dtype=torch.long).to(device)
labels_subset = labels[:N]

# 4. Вычисляем эмбеддинги [CLS]
batch_size = 100
features = []

with torch.no_grad():
    for i in tqdm(range(0, N, batch_size), desc="Computing BERT embeddings"):
        input_batch = input_ids[i:i+batch_size]
        attention_batch = attention_mask_tensor[i:i+batch_size]

        output = model(input_batch, attention_mask=attention_batch)
        cls_embeddings = output.last_hidden_state[:, 0, :]  # [CLS]-токен
        features.append(cls_embeddings.cpu())

# 5. Собираем итоговый массив признаков
features = torch.cat(features).numpy()


Computing BERT embeddings: 100%|██████████| 100/100 [04:24<00:00,  2.64s/it]


Теперь можем обучить модель логистической регрессии на эмбеддингах, которые мы получили в переменной features, и вычислить F1-меру на отложенной тестовой выборке.

In [9]:
# Деление данных (train → train+val и test отдельно)
# Сначала отделим 20% на тест
X_temp, X_test, y_temp, y_test = train_test_split(
    features, labels_subset, test_size=0.2, stratify=labels_subset, random_state=42
)

# Настроим пайплайн и подбор по train+val
# Пайплайн (простой, так как фичи уже готовы)
pipeline = Pipeline([
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced'))
])

# Сетка гиперпараметров
param_grid = {
    'clf__C': [0.1, 0.5, 1.0, 2.0]
}

# Стратифицированная кросс-валидация
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Поиск по сетке
grid = GridSearchCV(pipeline, param_grid, scoring='f1', cv=cv, n_jobs=-1, verbose=1)
grid.fit(X_temp, y_temp)

print("Лучшие параметры:", grid.best_params_)
print("F1 на кросс-валидации:", grid.best_score_)

# Финальная честная оценка на тестовой выборке
# Предсказания на тесте
y_test_pred = grid.predict(X_test)
f1_test = f1_score(y_test, y_test_pred)

print(f"\nФинальная F1-мера на тестовой выборке: {f1_test:.3f}")


Fitting 3 folds for each of 4 candidates, totalling 12 fits
Лучшие параметры: {'clf__C': 0.5}
F1 на кросс-валидации: 0.5487757106645126

Финальная F1-мера на тестовой выборке: 0.569


Вывод:

Результат F1-мера на тестовой выборке: 0.569 показывает, что использование "замороженного" BERT в качестве эмбеддера (без дообучения) и последующей классификации с помощью LogisticRegression даёт умеренное качество на небольшой выборке (10 000 примеров).

Причина — эмбеддинги BERT без дообучения не адаптированы под задачу классификации токсичности.

Поэтому следующий шаг — fine-tuning BERT, чтобы модель могла адаптироваться к задаче и значительно повысить F1-меру.

In [10]:
# 1. Dataset-класс
class ToxicCommentsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_len)
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

    def __len__(self):
        return len(self.labels)

# 2. Загрузка и подготовка данных
N = 5000  # для стабильности на CPU
texts = df['text'][:N].tolist()
labels = df['toxic'][:N].tolist()

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# 3. Разделение
X_train, X_val, y_train, y_val = train_test_split(texts, labels, test_size=0.2, random_state=42)

# 4. Dataset и DataLoader
train_dataset = ToxicCommentsDataset(X_train, y_train, tokenizer)
val_dataset = ToxicCommentsDataset(X_val, y_val, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)  # уменьшенный batch_size
val_loader = DataLoader(val_dataset, batch_size=8)

# 5. Модель и устройство
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")  # для Mac с M чипами Apple
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
model.to(device)

# 6. Оптимизатор
optimizer = AdamW(model.parameters(), lr=2e-5)

# 7. Цикл обучения
EPOCHS = 2  # сократим до 2 эпох для ускорения тестирования
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    
    for batch in loop:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    
    print(f"Epoch {epoch+1} loss: {total_loss / len(train_loader):.4f}")

# 8. Оценка
model.eval()
y_pred, y_true = [], []

with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        y_pred.extend(predictions.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

f1 = f1_score(y_true, y_pred)
print(f"\nF1-мера на валидационной выборке: {f1:.3f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|██████████| 500/500 [01:30<00:00,  5.52it/s, loss=0.0212] 


Epoch 1 loss: 0.2147


Epoch 2: 100%|██████████| 500/500 [01:26<00:00,  5.76it/s, loss=0.00878]


Epoch 2 loss: 0.1236

F1-мера на валидационной выборке: 0.715


Вывод:

Модель BERT уже на двух эпохах показала F1 = 0.715, что приближает нас к результатам, которых мы достигали ранее при помощи классических моделей (LogisticRegression, LinearSVC). Это подтверждает, что fine-tuning BERT работает эффективно при умеренном размере данных.

Дальше запустим оптимизированный код для fine-tuning BERT.

Код включает улучшения:

StratifiedShuffleSplit

scheduler для управления learning rate

F1-оценку после каждой эпохи

EPOCHS = 3

N = 10000

In [11]:
# 1. Класс Dataset
class ToxicCommentsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer.encode_plus(
            self.texts[idx],
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# 2. Загрузка и подготовка данных
df = pd.read_csv('/Users/dmitrysergeenko/Downloads/Яндекс.Практикум/Проекты на зачет/BERT. Векторизация текста/toxic_comments.csv')  # путь к датасету
texts = df['text'].astype(str).tolist()
labels = df['toxic'].tolist()

# 3. Ограничим объём
N = 10000
texts = texts[:N]
labels = labels[:N]

# 4. Токенизатор
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# 5. Разделение на train/val
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, val_idx in sss.split(texts, labels):
    X_train = [texts[i] for i in train_idx]
    X_val = [texts[i] for i in val_idx]
    y_train = [labels[i] for i in train_idx]
    y_val = [labels[i] for i in val_idx]

# 6. Dataset & DataLoader
train_dataset = ToxicCommentsDataset(X_train, y_train, tokenizer)
val_dataset = ToxicCommentsDataset(X_val, y_val, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# 7. Модель и устройство
device = torch.device("mps" if torch.backends.mps.is_available()
                      else "cuda" if torch.cuda.is_available()
                      else "cpu")

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
model.to(device)

# 8. Оптимизатор и scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
num_training_steps = len(train_loader) * 3
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

# 9. Обучение с отслеживанием лучшей F1
EPOCHS = 3
best_f1 = 0
best_epoch = 0
best_model_path = "best_bert_model.pt"

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        total_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"\nEpoch {epoch+1} average loss: {avg_loss:.4f}")

    # Валидация
    model.eval()
    y_pred = []
    y_true = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)

            y_pred.extend(predictions.cpu().numpy())
            y_true.extend(labels.cpu().numpy())

    f1 = f1_score(y_true, y_pred)
    print(f"F1-мера на валидационной выборке после эпохи {epoch+1}: {f1:.4f}")

    # Сохраняем лучшую модель
    if f1 > best_f1:
        best_f1 = f1
        best_epoch = epoch + 1
        torch.save(model.state_dict(), best_model_path)
        print(f"Лучшая модель обновлена на эпохе {epoch+1}")

# 10. Финальный вывод
print(f"\nФинальная F1-мера (последняя эпоха): {f1:.4f}")
print(f"Лучшая F1-мера: {best_f1:.4f} на эпохе {best_epoch}")

# 11. Загрузка лучшей модели (опционально перед оценкой на тесте)
model.load_state_dict(torch.load(best_model_path))
model.eval()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|██████████| 500/500 [02:28<00:00,  3.36it/s, loss=0.107]  



Epoch 1 average loss: 0.1805
F1-мера на валидационной выборке после эпохи 1: 0.7657
Лучшая модель обновлена на эпохе 1


Epoch 2: 100%|██████████| 500/500 [02:27<00:00,  3.39it/s, loss=0.194]  



Epoch 2 average loss: 0.0927
F1-мера на валидационной выборке после эпохи 2: 0.7749
Лучшая модель обновлена на эпохе 2


Epoch 3: 100%|██████████| 500/500 [02:27<00:00,  3.40it/s, loss=0.00235]



Epoch 3 average loss: 0.0431
F1-мера на валидационной выборке после эпохи 3: 0.7846
Лучшая модель обновлена на эпохе 3

Финальная F1-мера (последняя эпоха): 0.7846
Лучшая F1-мера: 0.7846 на эпохе 3


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

Результаты дообучения модели BERT:

**Модель:** `bert-base-multilingual-cased`  
**Задача:** Классификация токсичных комментариев (2 класса)  
**Размер обучающей выборки:** 10 000 примеров  
**Оптимизатор:** AdamW (learning rate = 2e-5)  
**Batch size:** 16  
**Эпохи обучения:** 3  
**Устройство:** Mac M4 Pro (MPS backend)

---

Финальный результат:
**F1-мера на валидационной выборке: 0.7846**

---

Вывод:

Fine-tuning предобученной BERT-модели позволил существенно превзойти классические модели (Logistic Regression и LinearSVC), улучшив F1-метрику с ~0.76 до **0.7846**.  
Это подтверждает высокую эффективность трансформеров в задачах классификации текста при наличии разметки и сбалансированного обучения.


## Общий вывод по проекту

В рамках проекта по классификации токсичных комментариев для платформы Викишоп были реализованы и сравнены два подхода:

1. Базовые модели машинного обучения:
- Построены пайплайны на основе `TfidfVectorizer` и моделей `LogisticRegression` и `LinearSVC`.
- Проведён подбор гиперпараметров с помощью `GridSearchCV`.
- Лучшая F1-мера, достигнутая при использовании модели `LinearSVC` с оптимизированными параметрами, составила **0.761**.

2. Дообучение трансформера BERT:
- Использована предобученная модель `bert-base-multilingual-cased` с дополнительным классификационным слоем (`BertForSequenceClassification`).
- Проведён fine-tuning на первых 10 000 примерах корпуса.
- Модель обучалась 3 эпохи:
  - Эпоха 1: 0.7657
  - Эпоха 2: 0.7749
  - Эпоха 3: **0.7846**
- Это превосходит результаты классических моделей, что подтверждает эффективность использования трансформеров для задачи обработки естественного языка.

Вывод:

Fine-tuning модели BERT позволяет достичь более высокой точности в задаче классификации токсичных комментариев. Использование современных языковых моделей оправдано даже при ограниченном количестве примеров и без применения GPU. Результаты подтверждают перспективность BERT-подхода для внедрения в реальный продукт Викишоп.
